In [1]:
# Install dependencies - não é necessário correr
import sys
!{sys.executable} -m pip install hdfs

In [2]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType, DoubleType
from pyspark.sql.functions import when, col, concat, lit

client = InsecureClient("http://hdfs-nn:9870", user="anonymous")

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/fisico/silver/'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-589cffad-fe9f-4fbb-8f14-004386978bb5;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 247ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      d

In [6]:
# apagar tudo do hdfs

#to_path = "/projeto/fisico/bronze/stats/"
#to_path_aux = "/projeto/fisico/bronze/players_stats_by_season_full_details/"
#to_path_aux_teams = "/projeto/fisico/bronze/team_abbreviations/"
#to_path_aux_age = "/projeto/fisico/bronze/age/"
to_path_aux_hist = "/projeto/fisico/bronze/historical/"

#to_path_silver = "/projeto/fisico/silver/stats/"


#client.delete(to_path,True)
#client.delete(to_path_aux,True)
#client.delete(to_path_aux_teams,True)
#client.delete(to_path_aux_age,True)
#client.delete(to_path_aux_hist,True)

#client.delete(to_path_silver,True)

In [4]:
# declaracao de todos os caminhos

from_path = "../projeto/bronze/stats/"
from_path_aux = "../projeto/bronze/players_stats_by_season_full_details/"
from_path_aux_teams = "../projeto/bronze/team_abbreviations/"
from_path_aux_age = "../projeto/bronze/age/"
from_path_aux_hist = "../projeto/bronze/historical/"

hdfs_path = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/stats"
hdfs_path_aux = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/players_stats_by_season_full_details"
hdfs_path_aux_teams = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/team_abbreviations"
hdfs_path_aux_age1718 = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/age/age_1718.csv"
hdfs_path_aux_age1819 = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/age/age_1819.csv"
hdfs_path_aux_age1920 = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/age/age_1920.csv"
hdfs_path_aux_hist = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/historical"

In [7]:
# upload de bronze

#client.upload(to_path, from_path)
#client.upload(to_path_aux, from_path_aux)
#client.upload(to_path_aux_teams, from_path_aux_teams)
#client.upload(to_path_aux_age, from_path_aux_age)
client.upload(to_path_aux_hist, from_path_aux_hist)

'/projeto/fisico/bronze/historical'

In [8]:
# criacao dos dataframes

dataFrame = spark.read.csv(hdfs_path, header= True)
dataFrame_aux = spark.read.csv(hdfs_path_aux, header= True)
dataFrame_aux_teams = spark.read.csv(hdfs_path_aux_teams, header= True)
dataFrame_aux_age1718 = spark.read.csv(hdfs_path_aux_age1718, header= True)
dataFrame_aux_age1819 = spark.read.csv(hdfs_path_aux_age1819, header= True)
dataFrame_aux_age1920 = spark.read.csv(hdfs_path_aux_age1920, header= True)
dataFrame_aux_hist = spark.read.csv(hdfs_path_aux_hist, header= True)

In [9]:
# visualizacao dos dataframes - alterar confome necessário

dataFrame.printSchema()
dataFrame_aux.printSchema()
dataFrame_aux_teams.printSchema()
#dataFrame.show(truncate=False)
#dataFrame.toPandas()

root
 |-- #: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Tm: string (nullable = true)
 |-- G: string (nullable = true)
 |-- GS: string (nullable = true)
 |-- MP: string (nullable = true)
 |-- PER: string (nullable = true)
 |-- TS%: string (nullable = true)
 |-- 3PAr: string (nullable = true)
 |-- FTr: string (nullable = true)
 |-- ORB%: string (nullable = true)
 |-- DRB%: string (nullable = true)
 |-- TRB%: string (nullable = true)
 |-- AST%: string (nullable = true)
 |-- STL%: string (nullable = true)
 |-- BLK%: string (nullable = true)
 |-- TOV%: string (nullable = true)
 |-- USG%: string (nullable = true)
 |-- blanl: string (nullable = true)
 |-- OWS: string (nullable = true)
 |-- DWS: string (nullable = true)
 |-- WS: string (nullable = true)
 |-- WS/48: string (nullable = true)
 |-- blank2: string (nullable = true)
 |-- OBPM: string (nullable = true)
 |

In [ ]:
# visualizacao estatistica dos dataframes - alterar conforme necessário

dataFrame_status = dataFrame.summary()
#dataFrame_aux_status = dataFrame_aux.summary()
#dataFrame_aux_teams_status = dataFrame_aux_teams.summary()

dataFrame_status.toPandas()
#dataFrame_aux_status.toPandas()
#dataFrame_aux_teams_status.toPandas()

In [ ]:
# edição do dataframe das idades

In [ ]:
df1718 = dataFrame_aux_age1718.withColumn(
    "Year",
    lit("2018")
    )
df1819 = dataFrame_aux_age1819.withColumn(
    "Year",
    lit("2019")
    )
df1920 = dataFrame_aux_age1920.withColumn(
    "Year",
    lit("2020")
    )

In [ ]:
# junção dos 3

df_aux_age = df1718.union(df1819.union(df1920))
df_aux_age = df_aux_age.drop("Team")
df_aux_age = df_aux_age.withColumnRenamed("Player","Player1")
df_aux_age = df_aux_age.withColumnRenamed("Year","Year1")

In [ ]:
# join do df das idades ao aux

In [ ]:
# inicio da edição dos dataframes

In [ ]:
# adicionar nomes das equipas por extenso

dataFrame = dataFrame.join(dataFrame_aux_teams, dataFrame.Tm == dataFrame_aux_teams.Abbreviation, 'leftouter')

dataFrame_aux = dataFrame_aux.join(dataFrame_aux_teams, dataFrame_aux.Team == dataFrame_aux_teams.Abbreviation, 'leftouter')

In [ ]:
# adaptacao do df aux ao df principal

In [ ]:
dataFrame_aux = dataFrame_aux.withColumn(    
    "Year",
    dataFrame_aux.Season.substr(8, 4)
)

In [ ]:
cond = [dataFrame_aux.Year == df_aux_age.Year1, dataFrame_aux.Player == df_aux_age.Player1]

dataFrame_aux = dataFrame_aux.join(df_aux_age, cond, 'leftouter')

In [ ]:
dataFrame_aux.createOrReplaceTempView("dataFrame_aux")

# preparar aux para a juncao
dataFrame_aux = spark.sql(
    """
    SELECT Year,Player,Age,Team,GP,MIN,FGM,FGA,3PM,3PA,FTM,FTA,TOV,PF,ORB,DRB,REB,AST,STL,BLK,PTS,Franchise
    FROM dataFrame_aux
    WHERE (Year == "2018" OR Year == "2019" OR Year == "2020")
    AND League == "NBA"
    AND Stage == "Regular_Season"
    """
)

dataFrame_aux.show()

In [ ]:
# adicionar FG%, eFG%, 3P%

df_aux = dataFrame_aux.withColumn(    
    "FG%",
    col("FGM") / col("FGA")
) \
                .withColumn(    
    "eFG%",
    (col("FGM") + 0.5 * col("3PM")) / col("FGA")
) \
                .withColumn(    
    "3P%",
    col("3PM") / col("3PA")
)

In [ ]:
# igualar os nomes do df_aux aos do principal

df_aux = df_aux.withColumnRenamed('Team', 'Tm') \
                .withColumnRenamed('Franchise', 'Team') \
                .withColumnRenamed('GP', 'G') \
                .withColumnRenamed('MIN', 'MP') \
                .withColumnRenamed('FGM', 'FG') \
                .withColumnRenamed('3PM', '3P') \
                .withColumnRenamed('FTM', 'FT') \
                .withColumnRenamed('GP', 'G') \
                .withColumnRenamed('REB', 'TRB')

In [ ]:
# reordenar df para ficar igual ao principal - prob não é necessário mas wtv
df_aux = df_aux.select("Year","Player","Age","Tm","Team","G","MP","FG","FGA","FG%","3P","3PA","3P%","FT","FTA","TOV","PF","ORB","DRB","TRB","AST","STL","BLK","PTS")

In [ ]:
df_aux.printSchema()
#df_aux.show(truncate=False)
#df_aux.toPandas()

In [ ]:
# EDIÇÃO DO DATAFRAME PRINCIPAL

In [ ]:
# apagar o indice porque o toPandas já mostra indice (honestamente é porque facilita a alteração de baixo lmao)
df = dataFrame.drop('#')

# apagar blanl e blank2 porque sao colunas com todas as linhas null (começo a pensar que este dataset não é muito bom xd)
df = df.drop('blanl')
df = df.drop('blank2')

In [ ]:
# apagar as 67 linhas que estão em branco
df = df.na.drop("all")

In [ ]:
# team faz mais sentido do que franchise
df = df.withColumnRenamed('Franchise', 'Team')

### edicao aux_historical

In [ ]:
df_hist.printSchema()
df_hist.toPandas()

In [ ]:
df_hist = dataFrame_aux_hist.withColumn(    
    "Wins",
    dataFrame_aux_hist.Record.substr(1, 2)
) \
.withColumn(    
    "Losses",
    dataFrame_aux_hist.Record.substr(4, 2)
)

In [ ]:
df_hist = df_hist.withColumn(    
    "Year_fixed",
    df_hist.Year.substr(6, 2).cast(IntegerType())
)

In [ ]:
df_hist20 = df_hist.filter("Year_fixed < 50")

In [ ]:
df_hist19 = df_hist.filter("Year_fixed > 50")

In [ ]:
from pyspark.sql import functions as func

df_hist20 = df_hist20.withColumn(    
    "Year_fixed",
    func.concat(lit("20"),func.col('Year_fixed'))
)

df_hist19 = df_hist19.withColumn(    
    "Year_fixed",
    func.concat(lit("19"),func.col('Year_fixed'))
)

In [ ]:
df_hist_final = df_hist20.unionByName(df_hist19, allowMissingColumns = True)

In [ ]:
df_hist_final = df_hist_final.select("Year_fixed","Tm","Wins","Losses","Winning Percentage")

In [ ]:
df_hist_final = df_hist_final.withColumnRenamed("Tm","TmA")

In [ ]:
df_hist_final.printSchema()
df_hist_final.toPandas()

In [ ]:
### juncao do hist ao complete

# JUNCAO DOS DOIS DATAFRAMES

In [ ]:
df_complete = df.unionByName(df_aux, allowMissingColumns = True)

In [ ]:
### juncao do hist ao complete

In [ ]:
cond = [df_complete.Year == df_hist_final.Year_fixed, df_complete.Tm == df_hist_final.TmA]

df_complete = df_complete.join(df_hist_final, cond, 'leftouter')

In [ ]:
df_complete.filter("Wins > 0").toPandas()

In [ ]:
df_complete = df_complete.withColumn("Year", df_complete["Year"].cast(StringType())) 
df_complete = df_complete.withColumn("Player", df_complete["Player"].cast(StringType()))
df_complete = df_complete.withColumn("Pos", df_complete["Pos"].cast(StringType())) 
df_complete = df_complete.withColumn("Age", df_complete["Age"].cast(IntegerType()))
df_complete = df_complete.withColumn("Tm", df_complete["Tm"].cast(StringType())) 
df_complete = df_complete.withColumn("G", df_complete["G"].cast(IntegerType())) 
df_complete = df_complete.withColumn("GS",df_complete["GS"].cast(IntegerType())) 
df_complete = df_complete.withColumn("MP", df_complete["MP"].cast(IntegerType())) 
df_complete = df_complete.withColumn("PER", df_complete["PER"].cast(DoubleType()))
df_complete = df_complete.withColumn("TSP", df_complete["TS%"].cast(DoubleType()))
df_complete = df_complete.withColumn("3PAr", df_complete["3PAr"].cast(DoubleType()))
df_complete = df_complete.withColumn("FTr", df_complete["FTr"].cast(DoubleType()))
df_complete = df_complete.withColumn("ORBP", df_complete["ORB%"].cast(DoubleType()))
df_complete = df_complete.withColumn("DRBP", df_complete["DRB%"].cast(DoubleType()))
df_complete = df_complete.withColumn("TRBP", df_complete["TRB%"].cast(DoubleType()))
df_complete = df_complete.withColumn("ASTP", df_complete["AST%"].cast(DoubleType()))
df_complete = df_complete.withColumn("STLP", df_complete["STL%"].cast(DoubleType()))
df_complete = df_complete.withColumn("BLKP", df_complete["BLK%"].cast(DoubleType()))
df_complete = df_complete.withColumn("TOVP", df_complete["TOV%"].cast(DoubleType()))
df_complete = df_complete.withColumn("USGP", df_complete["USG%"].cast(DoubleType()))
df_complete = df_complete.withColumn("OWS", df_complete["OWS"].cast(DoubleType()))
df_complete = df_complete.withColumn("DWS", df_complete["DWS"].cast(DoubleType()))
df_complete = df_complete.withColumn("WS", df_complete["WS"].cast(DoubleType()))
df_complete = df_complete.withColumn("WS48", df_complete["WS/48"].cast(DoubleType()))
df_complete = df_complete.withColumn("OBPM", df_complete["OBPM"].cast(DoubleType()))
df_complete = df_complete.withColumn("DBPM", df_complete["DBPM"].cast(DoubleType()))
df_complete = df_complete.withColumn("BPM", df_complete["BPM"].cast(DoubleType()))
df_complete = df_complete.withColumn("VORP", df_complete["VORP"].cast(DoubleType()))
df_complete = df_complete.withColumn("FG", df_complete["FG"].cast(DoubleType()))
df_complete = df_complete.withColumn("OBPM", df_complete["OBPM"].cast(DoubleType()))
df_complete = df_complete.withColumn("DBPM", df_complete["DBPM"].cast(DoubleType()))
df_complete = df_complete.withColumn("VORP", df_complete["VORP"].cast(DoubleType()))
df_complete = df_complete.withColumn("FG", df_complete["FG"].cast(DoubleType()))
df_complete = df_complete.withColumn("FGA", df_complete["FGA"].cast(DoubleType()))
df_complete = df_complete.withColumn("FGP", df_complete["FG%"].cast(DoubleType()))
df_complete = df_complete.withColumn("3P", df_complete["3P"].cast(DoubleType()))
df_complete = df_complete.withColumn("3PA", df_complete["3PA"].cast(DoubleType()))
df_complete = df_complete.withColumn("3PP", df_complete["3P%"].cast(DoubleType()))
df_complete = df_complete.withColumn("2P", df_complete["2P"].cast(DoubleType()))
df_complete = df_complete.withColumn("2PA", df_complete["2PA"].cast(DoubleType()))
df_complete = df_complete.withColumn("2PP", df_complete["2P%"].cast(DoubleType()))
df_complete = df_complete.withColumn("eFGP", df_complete["eFG%"].cast(DoubleType()))
df_complete = df_complete.withColumn("FT", df_complete["FT"].cast(DoubleType()))
df_complete = df_complete.withColumn("FTA", df_complete["FTA"].cast(DoubleType()))
df_complete = df_complete.withColumn("FTP", df_complete["FT%"].cast(DoubleType()))
df_complete = df_complete.withColumn("ORB", df_complete["ORB"].cast(DoubleType()))
df_complete = df_complete.withColumn("DRB", df_complete["DRB"].cast(DoubleType()))
df_complete = df_complete.withColumn("TRB", df_complete["TRB"].cast(DoubleType()))
df_complete = df_complete.withColumn("AST", df_complete["AST"].cast(DoubleType()))
df_complete = df_complete.withColumn("STL", df_complete["STL"].cast(DoubleType()))
df_complete = df_complete.withColumn("BLK", df_complete["BLK"].cast(DoubleType()))
df_complete = df_complete.withColumn("TOV", df_complete["TOV"].cast(DoubleType()))
df_complete = df_complete.withColumn("PF", df_complete["PF"].cast(DoubleType()))
df_complete = df_complete.withColumn("PTS", df_complete["PTS"].cast(DoubleType()))
#df_complete = df_complete.withColumn("Abbreviation", df_complete["Abbreviation"].cast(StringType()))
df_complete = df_complete.withColumn("Team", df_complete["Team"].cast(StringType()))

df_complete = df_complete.withColumn("Wins", df_complete["Wins"].cast(IntegerType()))
df_complete = df_complete.withColumn("Losses", df_complete["Losses"].cast(IntegerType()))
df_complete = df_complete.withColumn("WP", df_complete["Winning Percentage"].cast(DoubleType()))

In [ ]:
df_complete.printSchema()
#df_complete.show(truncate=False)
#df_complete.toPandas()
#topandas usar apenas para mostrar os dados em tabela

In [ ]:
df_complete \
    .select("Year","Player","Pos","Age","Tm","Team","G","GS","Wins","Losses","WP","MP","PER","TSP","3PAr","FTr","ORBP","DRBP","TRBP","ASTP","STLP","BLKP","TOVP","USGP","OWS","DWS","WS","WS48","OBPM","DBPM","BPM","VORP","FG","FGA","FGP","3P","3PA","3PP","2P","2PA","2PP","eFGP","FT","FTA","FTP","ORB","DRB","TRB","AST","STL","BLK","TOV","PF","PTS") \
    .write \
    .mode("overwrite") \
    .partitionBy("Year") \
    .format("delta") \
    .save("hdfs://hdfs-nn:9000/projeto/fisico/silver/stats")